In [30]:
from maap.maap import MAAP
maap = MAAP(maap_host='api.ops.maap-project.org')

# Launch DPS for mapBoreal.R

In [31]:
import os
import geopandas
import pandas as pd
import json

def local_to_s3(url, user='lduncanson'):
    ''' A Function to convert local paths to s3 urls'''
    return url.replace('/projects/my-private-bucket', f's3://maap-ops-workspace/{user}')
def local_to_https(url, user='lduncanson'):
    ''' A Function to convert local paths to https urls'''
    return url.replace('/projects/my-private-bucket', f'https://s3.console.aws.amazon.com/s3/buckets/maap-ops-workspace/{user}')
def local_to_https_uswest2(url, user='lduncanson'):
    ''' A Function to convert local paths to https us-west-s urls'''
    return url.replace('/projects/my-private-bucket', f'https://maap-ops-workspace.s3.us-west-2.amazonaws.com/{user}')

In [32]:
atl08_filt_tindex_master =   pd.read_csv('s3://maap-ops-workspace/shared/lduncanson/DPS_tile_lists/ATL08_filt_tindex_master.csv')
topo_tindex_master =         pd.read_csv('s3://maap-ops-workspace/shared/nathanmthomas/DPS_tile_lists/Topo_tindex_master.csv')
landsat_tindex_master =      pd.read_csv('s3://maap-ops-workspace/shared/nathanmthomas/DPS_tile_lists/Landsat_tindex_master.csv')

# Convert al local_paths to s3
#.. for data produced by 'lduncanson' workspace
atl08_filt_tindex_master['https'] = [local_to_https_uswest2(local_path, user='lduncanson') for local_path in atl08_filt_tindex_master['local_path']]

#.. for data produced by 'nathanmthomas' workspace
for tindex_master in [topo_tindex_master, landsat_tindex_master]:
    tindex_master['https'] = [local_to_https_uswest2(local_path, user='nathanmthomas') for local_path in tindex_master['local_path']]

# Use the ATL08 filtered tindex master list to tell you which tiles you'll run

In [33]:
INPUT_TILE_NUM_LIST = atl08_filt_tindex_master['tile_num'].values.astype(int).tolist()
print(len(INPUT_TILE_NUM_LIST))

# Remove duplicate tile nums
INPUT_TILE_NUM_LIST = list(set(INPUT_TILE_NUM_LIST))

print(len(INPUT_TILE_NUM_LIST))

3968
3940


In [34]:
# Check retrieval of s3 path with a tle_num
in_atl08_https =  atl08_filt_tindex_master['https'].loc[atl08_filt_tindex_master['tile_num'] == INPUT_TILE_NUM_LIST[0]].tolist()[0]
in_topo_https = topo_tindex_master['https'].loc[topo_tindex_master['tile_num'] == INPUT_TILE_NUM_LIST[0]].tolist()[0]
in_landsat_https = landsat_tindex_master['https'].loc[landsat_tindex_master['tile_num'] == INPUT_TILE_NUM_LIST[0]].tolist()[0]
print(in_atl08_https)
print(in_topo_https)
print(in_landsat_https)

https://maap-ops-workspace.s3.us-west-2.amazonaws.com/lduncanson/dps_output/run_tile_atl08_ubuntu/master/2021/10/09/05/15/08/304917/atl08_004_30m_filt_topo_landsat_20211009_0009.csv
https://maap-ops-workspace.s3.us-west-2.amazonaws.com/nathanmthomas/dps_output/do_topo_stack_3-1-5_ubuntu/ops/2021/09/15/18/10/34/658640/Copernicus_9_covars_cog_topo_stack.tif
https://maap-ops-workspace.s3.us-west-2.amazonaws.com/nathanmthomas/dps_output/do_landsat_stack_3-1-2_ubuntu/ops/2021/09/14/19/20/02/503587/Landsat8_9_comp_cog_2015-2020_dps.tif


## Get files for boreal biomass models & boreal wide sample


In [35]:
bio_models_https = 'https://s3.console.aws.amazon.com/s3/buckets/maap-ops-workspace/lduncanson/my-private-bucket/bio_models.tar'

train_data_https = 'https://maap-ops-workspace.s3.us-west-2.amazonaws.com/lduncanson/boreal_train_data_v2.csv'

## Run a DPS job

In [36]:
RUN_DPS  = True

if RUN_DPS:
    ##################################
    #Test DPS submission on a single file
    #for i, INPUT_TILE_NUM in enumerate(INPUT_TILE_NUM_LIST):
    for i, INPUT_TILE_NUM in enumerate(INPUT_TILE_NUM_LIST):
        DPS_num = i+1
        
        # Get the s3 paths of the corresponding input filenames with an input tile_num
        in_atl08_https =  atl08_filt_tindex_master['https'].loc[atl08_filt_tindex_master['tile_num'] == INPUT_TILE_NUM].tolist()[0]
        in_topo_https = topo_tindex_master['https'].loc[topo_tindex_master['tile_num'] == INPUT_TILE_NUM].tolist()[0]
        in_landsat_https = landsat_tindex_master['https'].loc[landsat_tindex_master['tile_num'] == INPUT_TILE_NUM].tolist()[0]
        
        #print(in_atl08_https) 
        #print(in_topo_https)
        #print(in_landsat_https)
        
        if True:
            in_param_dict = {
                                    'in_atl08_fn': f"input/{os.path.basename(in_atl08_https)}",
                                    'in_topo_fn': f"input/{os.path.basename(in_topo_https)}",
                                    'in_landsat_fn': f"input/{os.path.basename(in_landsat_https)}",
                                    'in_atl08_fn_url': in_atl08_https,
                                    'in_topo_fn_url': in_topo_https,
                                    'in_landsat_fn_url': in_landsat_https,
                                    'DO_SLOPE_VALID_MASK': 'TRUE',
                                    'in_atl08_sample_fn':f"input/{os.path.basename(train_data_https)}",
                                    'in_atl08_sample_url':train_data_https,
                                    'in_tile_num': INPUT_TILE_NUM,
                                    'in_tile_fn_url': 'https://maap-ops-workspace.s3.amazonaws.com/shared/nathanmthomas/boreal_grid_albers90k_gpkg.gpkg',
                                    'in_tile_fn': 'input/boreal_grid_albers90k_gpkg.gpkg'
                }

            submit_result = maap.submitJob(
                    identifier='run_boreal_biomass',
                    algo_id='run_boreal_biomass_ubuntu',
                    version='master',
                    username='lduncanson', # username needs to be the same as whoever created the workspace
                    queue='maap-dps-worker-32gb',
                    **in_param_dict
                )

            #submit_result = 'submit test'
            if DPS_num in [1, 5, 10, 15, 20, 25, 50, 75, 100,200, 500,1000,2000, 3000,3500, 4000, len(INPUT_TILE_NUM_LIST)]:
                print(f"DPS run num: {DPS_num}, tile num: {INPUT_TILE_NUM}, job info: {submit_result}") 
                print(f"DPS job status: {maap.getJobStatus(submit_result.get('job_id')) }" )
        else:
            print(f"Tile num: {INPUT_TILE_NUM}")
 

DPS run num: 1, tile num: 9, job info: {'status': 'success', 'http_status_code': 200, 'job_id': '6575a64d-2506-4faf-93d4-7ea88ea12ba4'}
DPS job status: <Response [200]>
DPS run num: 5, tile num: 17, job info: {'status': 'success', 'http_status_code': 200, 'job_id': 'bf9efeb2-7cfa-4588-ab5c-3f7899fb3c23'}
DPS job status: <Response [200]>
DPS run num: 10, tile num: 25, job info: {'status': 'success', 'http_status_code': 200, 'job_id': 'b43e6b3d-6787-42c0-8d7b-8d6c2d3f3a74'}
DPS job status: <Response [200]>
DPS run num: 15, tile num: 30, job info: {'status': 'success', 'http_status_code': 200, 'job_id': '24c93ba4-3afd-46c2-8521-f8e61ad9259a'}
DPS job status: <Response [200]>
DPS run num: 20, tile num: 37, job info: {'status': 'success', 'http_status_code': 200, 'job_id': '2f334866-6ac7-49e6-81f5-e670b423f6cb'}
DPS job status: <Response [200]>
DPS run num: 25, tile num: 42, job info: {'status': 'success', 'http_status_code': 200, 'job_id': '094eb6c0-d674-4e39-8351-479a93488907'}
DPS job st

## Get another list just of missing tiles

In [29]:
need_tindex_master = pd.read_csv('/projects/my-public-bucket/DPS_tile_lists/Need_AGB_tindex_master.csv')
print(len(need_tindex_master))

INPUT_TILE_NUM_LIST = need_tindex_master.tile_num.tolist()

# Remove duplicate tile nums
INPUT_TILE_NUM_LIST = list(set(INPUT_TILE_NUM_LIST))

len(INPUT_TILE_NUM_LIST)

3482


3456